In [16]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [5]:
import pandas as pd
# Load the data
data = pd.read_csv('umsatz.csv')
data

,Kundengruppe,Artikelgruppe,Region,Umsatz
0,Fachhandel,Tourenräder,West,niedrig
1,Discounter,Cityräder,West,hoch
2,Fachhandel,Mountainbikes,Nord,mittel
3,Fachhandel,Tourenräder,Nord,niedrig
4,Fachhandel,Tourenräder,Süd,hoch
5,Kaufhaus,Mountainbikes,Nord,mittel
6,Discounter,Mountainbikes,West,mittel
7,Discounter,Cityräder,West,hoch


In [6]:
import math


# Function to calculate entropy for a given column
def entropy(column):
    counts = column.value_counts()
    probabilities = counts / counts.sum()
    return sum([-p * math.log2(p) for p in probabilities])

# Calculate total entropy
total_entropy = entropy(data['Umsatz'])

# Calculate information gain for each column
for column in ['Kundengruppe', 'Artikelgruppe', 'Region']:
    groups = data.groupby(column)
    weighted_entropies = groups.apply(lambda x: entropy(x['Umsatz']) * len(x) / len(data))
    information_gain = total_entropy - weighted_entropies.sum()
    print(f"Information gain for {column}: {information_gain:.3f}")


Information gain for Kundengruppe: 0.811
Information gain for Artikelgruppe: 1.217
Information gain for Region: 0.561


In [ ]:
inkonstenzRate = 0
for column in ['Kundengruppe', 'Artikelgruppe', 'Region']:
    #wenn die Ausprägung von 2 Merkmale gleich ist, dann berechnet man die inkonstenzRate,
    if inkonstenzRate == 0:
            #wenn die IR gleich 0, dann wird die gespeichert in eine Liste für weitere Berechnung, eine Zeile tiefer.
    if inkonstenzRate != 0:
            # wenn ungleich 0, dann gespeichert in eine andere Liste um zu abchecken, ob oben eine Zeile tiefer gegeangene eine Teilmenge davon ist.

    

In [15]:
# Encode categorical variables as numerical values
encoded_data = data.copy()
encoded_data['Kundengruppe'] = pd.factorize(data['Kundengruppe'])[0]
encoded_data['Artikelgruppe'] = pd.factorize(data['Artikelgruppe'])[0]
encoded_data['Region'] = pd.factorize(data['Region'])[0]
encoded_data

,Kundengruppe,Artikelgruppe,Region,Umsatz
0,0,0,0,niedrig
1,1,1,0,hoch
2,0,2,1,mittel
3,0,0,1,niedrig
4,0,0,2,hoch
5,2,2,1,mittel
6,1,2,0,mittel
7,1,1,0,hoch


In [27]:
import itertools

# Define a function to calculate inconsistency rate
def calculate_inconsistency_rate(df, target_col):
    target_vals = df[target_col].unique()
    print(target_vals)
    target_val_counts = df[target_col].value_counts()
    total_instances = len(df)
    inconsistency_rate = 0
    for val in target_vals:
        val_count = target_val_counts[val]
        val_fraction = val_count / total_instances
        inconsistency_rate += val_fraction * (1 - val_fraction)
    return inconsistency_rate

# Define a function to generate subsets of features
def generate_feature_subsets(features):
    subsets = []
    for L in range(0, len(features)+1):
        for subset in itertools.combinations(features, L):
            subsets.append(subset)
    return subsets[1:]

# Define the Branch and Bound algorithm
def branch_and_bound(df, features, target_col):
    # Initialize best subset and best inconsistency rate
    best_subset = features
    best_inconsistency_rate = 0 
    calculate_inconsistency_rate(df,target_col)
#     print(best_subset)
    
    
    
    # Generate initial node with all features
    initial_node = (features, calculate_inconsistency_rate(df, target_col))
    # Initialize the priority queue with the initial node
    priority_queue = [initial_node]
    print(priority_queue)
    while priority_queue:
        # Get the node with the highest priority (lowest inconsistency rate)
        current_node = priority_queue.pop(0)
        current_subset, current_inconsistency_rate = current_node
        # Check if current subset is better than the current best subset
        if current_inconsistency_rate <=best_inconsistency_rate:
            best_subset = current_subset
            best_inconsistency_rate = current_inconsistency_rate
        # Generate child nodes
        for feature in current_subset:
            child_subset = list(current_subset)
            child_subset.remove(feature)
            child_df = df.drop(columns=[feature])
            child_inconsistency_rate = calculate_inconsistency_rate(child_df, target_col)
            # Only add child nodes if their inconsistency rate is lower than the best inconsistency rate so far
            if child_inconsistency_rate < best_inconsistency_rate:
                child_node = (child_subset, child_inconsistency_rate)
                priority_queue.append(child_node)
                priority_queue.sort(key=lambda node: node[1])
    return best_subset, best_inconsistency_rate

In [28]:
# Call the Branch and Bound algorithm with the data and target column
features = ['Kundengruppe', 'Artikelgruppe', 'Region']
target_col = 'Umsatz'
best_subset, best_inconsistency_rate = branch_and_bound(data, features, target_col)

# Print the best subset and best inconsistency rate
print(f"Best subset: {best_subset}")
print(f"Best inconsistency rate: {best_inconsistency_rate}")


['niedrig' 'hoch ' 'mittel' 'mittel ']
['niedrig' 'hoch ' 'mittel' 'mittel ']
[(['Kundengruppe', 'Artikelgruppe', 'Region'], 0.71875)]
['niedrig' 'hoch ' 'mittel' 'mittel ']
['niedrig' 'hoch ' 'mittel' 'mittel ']
['niedrig' 'hoch ' 'mittel' 'mittel ']
Best subset: ['Kundengruppe', 'Artikelgruppe', 'Region']
Best inconsistency rate: 0
